In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

pd.set_option('display.max_colwidth', None)

In [2]:
customers_df = pd.read_csv("Customers.csv")
products_df = pd.read_csv("Products.csv")
transactions_df = pd.read_csv("Transactions.csv")

In [3]:
transactions_df = transactions_df.merge(products_df, on="ProductID", how="left")

# Aggregate customer purchase behavior
customer_purchases = transactions_df.groupby(["CustomerID", "Category"]).agg(
    {"Quantity": "sum", "TotalValue": "sum"}).unstack(fill_value=0)

customer_purchases

Quantity                                 TotalValue           \
Category      Books Clothing Electronics Home Decor      Books Clothing   
CustomerID                                                                
C0001             2        0           7          3     114.60     0.00   
C0002             0        4           0          6       0.00  1025.46   
C0003             0        4           4          6       0.00   122.36   
C0004             8        0           6          9    1888.48     0.00   
C0005             0        0           4          3       0.00     0.00   
...             ...      ...         ...        ...        ...      ...   
C0196             3        4           0          5    1310.67  1585.36   
C0197             0        0           6          3       0.00     0.00   
C0198             0        2           1          0       0.00   904.84   
C0199             0        0           3          6       0.00     0.00   
C0200             4        7           1          4    1665.60  2225.46   

                                   
Category   Electronics Home Decor  
CustomerID                         
C0001          2827.30     412.62  
C0002             0.00     837.28  
C0003          1385.20    1217.82  
C0004          1355.74    2110.66  
C0005          1180.38     853.86  
...                ...        ...  
C0196             0.00    2086.85  
C0197           914.92    1013.73  
C0198            26.99       0.00  
C0199           594.38    1384.90  
C0200           239.70     627.84  

[199 rows x 8 columns]

In [4]:
# Flatten the multi-index columns
customer_purchases.columns = ['_'.join(col).strip() for col in customer_purchases.columns.values]

customer_purchases

,Quantity_Books,Quantity_Clothing,Quantity_Electronics,Quantity_Home Decor,TotalValue_Books,TotalValue_Clothing,TotalValue_Electronics,TotalValue_Home Decor
CustomerID,,,,,,,,
C0001,2,0,7,3,114.60,0.00,2827.30,412.62
C0002,0,4,0,6,0.00,1025.46,0.00,837.28
C0003,0,4,4,6,0.00,122.36,1385.20,1217.82
C0004,8,0,6,9,1888.48,0.00,1355.74,2110.66
C0005,0,0,4,3,0.00,0.00,1180.38,853.86
...,...,...,...,...,...,...,...,...
C0196,3,4,0,5,1310.67,1585.36,0.00,2086.85
C0197,0,0,6,3,0.00,0.00,914.92,1013.73
C0198,0,2,1,0,0.00,904.84,26.99,0.00


In [5]:
# Merge with customer details
customer_profiles = customers_df.merge(customer_purchases, on="CustomerID", how="left").fillna(0)

customer_profiles

,CustomerID,CustomerName,Region,SignupDate,Quantity_Books,Quantity_Clothing,Quantity_Electronics,Quantity_Home Decor,TotalValue_Books,TotalValue_Clothing,TotalValue_Electronics,TotalValue_Home Decor
0,C0001,Lawrence Carroll,South America,2022-07-10,2.0,0.0,7.0,3.0,114.60,0.00,2827.30,412.62
1,C0002,Elizabeth Lutz,Asia,2022-02-13,0.0,4.0,0.0,6.0,0.00,1025.46,0.00,837.28
2,C0003,Michael Rivera,South America,2024-03-07,0.0,4.0,4.0,6.0,0.00,122.36,1385.20,1217.82
3,C0004,Kathleen Rodriguez,South America,2022-10-09,8.0,0.0,6.0,9.0,1888.48,0.00,1355.74,2110.66
4,C0005,Laura Weber,Asia,2022-08-15,0.0,0.0,4.0,3.0,0.00,0.00,1180.38,853.86
...,...,...,...,...,...,...,...,...,...,...,...,...
195,C0196,Laura Watts,Europe,2022-06-07,3.0,4.0,0.0,5.0,1310.67,1585.36,0.00,2086.85
196,C0197,Christina Harvey,Europe,2023-03-21,0.0,0.0,6.0,3.0,0.00,0.00,914.92,1013.73
197,C0198,Rebecca Ray,Europe,2022-02-27,0.0,2.0,1.0,0.0,0.00,904.84,26.99,0.00
198,C0199,Andrea Jenkins,Europe,2022-12-03,0.0,0.0,3.0,6.0,0.00,0.00,594.38,1384.90


In [6]:
# Convert categorical data (Region) into numerical using one-hot encoding
customer_profiles = pd.get_dummies(customer_profiles, columns=["Region"])

customer_profiles

,CustomerID,CustomerName,SignupDate,Quantity_Books,Quantity_Clothing,Quantity_Electronics,Quantity_Home Decor,TotalValue_Books,TotalValue_Clothing,TotalValue_Electronics,TotalValue_Home Decor,Region_Asia,Region_Europe,Region_North America,Region_South America
0,C0001,Lawrence Carroll,2022-07-10,2.0,0.0,7.0,3.0,114.60,0.00,2827.30,412.62,False,False,False,True
1,C0002,Elizabeth Lutz,2022-02-13,0.0,4.0,0.0,6.0,0.00,1025.46,0.00,837.28,True,False,False,False
2,C0003,Michael Rivera,2024-03-07,0.0,4.0,4.0,6.0,0.00,122.36,1385.20,1217.82,False,False,False,True
3,C0004,Kathleen Rodriguez,2022-10-09,8.0,0.0,6.0,9.0,1888.48,0.00,1355.74,2110.66,False,False,False,True
4,C0005,Laura Weber,2022-08-15,0.0,0.0,4.0,3.0,0.00,0.00,1180.38,853.86,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,C0196,Laura Watts,2022-06-07,3.0,4.0,0.0,5.0,1310.67,1585.36,0.00,2086.85,False,True,False,False
196,C0197,Christina Harvey,2023-03-21,0.0,0.0,6.0,3.0,0.00,0.00,914.92,1013.73,False,True,False,False
197,C0198,Rebecca Ray,2022-02-27,0.0,2.0,1.0,0.0,0.00,904.84,26.99,0.00,False,True,False,False
198,C0199,Andrea Jenkins,2022-12-03,0.0,0.0,3.0,6.0,0.00,0.00,594.38,1384.90,False,True,False,False


In [7]:
customer_ids = customer_profiles["CustomerID"]

customer_features = customer_profiles.drop(columns=["CustomerID", "CustomerName", "SignupDate"])

In [8]:
# Compute cosine similarity between all customers
similarity_matrix = cosine_similarity(customer_features)

similarity_df = pd.DataFrame(similarity_matrix, index=customer_ids, columns=customer_ids)

similarity_df.head(10)

CustomerID,C0001,C0002,C0003,C0004,C0005,C0006,C0007,C0008,C0009,C0010,...,C0191,C0192,C0193,C0194,C0195,C0196,C0197,C0198,C0199,C0200
CustomerID,,,,,,,,,,,,,,,,,,,,,
C0001,1.000000,0.091263,0.835989,0.548003,0.885670,0.085007,0.832408,0.448842,0.682405,0.008167,...,0.335834,0.969988,4.007840e-02,0.757178,0.476397,0.120692,7.695651e-01,0.029481,5.225496e-01,0.137893
C0002,0.091263,1.000000,0.467961,0.425135,0.370686,0.751093,0.422269,0.912573,0.560516,0.758337,...,0.000000,0.152377,2.510061e-07,0.668104,0.849546,0.869506,4.695053e-01,0.774246,5.811904e-01,0.741627
C0003,0.835989,0.467961,1.000000,0.766419,0.993310,0.310550,0.997755,0.724111,0.565115,0.064819,...,0.225568,0.746372,0.000000e+00,0.784649,0.847894,0.505021,9.911643e-01,0.088515,9.009729e-01,0.258967
C0004,0.548003,0.425135,0.766419,1.000000,0.743814,0.667393,0.770239,0.691921,0.298003,0.122555,...,0.703510,0.458666,6.014364e-01,0.625940,0.717383,0.747744,7.882965e-01,0.012875,7.879970e-01,0.534049
C0005,0.885670,0.370686,0.993310,0.743814,1.000000,0.238753,0.994511,0.655929,0.559215,0.000000,...,0.243888,0.792909,2.280969e-07,0.764901,0.781575,0.417414,9.779580e-01,0.024160,8.581494e-01,0.196586
C0006,0.085007,0.751093,0.310550,0.667393,0.238753,1.000000,0.271977,0.805650,0.460991,0.757028,...,0.624036,0.164628,6.543851e-01,0.625289,0.604385,0.927487,3.024018e-01,0.636773,3.743345e-01,0.966301
C0007,0.832408,0.422269,0.997755,0.770239,0.994511,0.271977,1.000000,0.683954,0.513819,0.000000,...,0.224089,0.728542,1.818927e-07,0.744220,0.826102,0.475500,9.944354e-01,0.022199,9.071570e-01,0.208979
C0008,0.448842,0.912573,0.724111,0.691921,0.655929,0.805650,0.683954,1.000000,0.725222,0.689603,...,0.290419,0.488976,1.927087e-01,0.886105,0.928741,0.893800,7.045809e-01,0.674541,7.180642e-01,0.797026
C0009,0.682405,0.560516,0.565115,0.298003,0.559215,0.460991,0.513819,0.725222,1.000000,0.708443,...,0.207753,0.817782,0.000000e+00,0.926139,0.547974,0.391510,4.624286e-01,0.743885,2.722089e-01,0.620964


In [9]:
# Function to get top 3 lookalikes for a given customer

def get_top_lookalikes(customer_id, top_n=3):
    similar_customers = similarity_df[customer_id].drop(customer_id).nlargest(top_n)

    return list(zip(similar_customers.index, similar_customers.values))

In [10]:
# Get top 3 lookalikes for all customers

lookalike_results = {cust_id: get_top_lookalikes(cust_id) for cust_id in customer_ids}

lookalike_df = pd.DataFrame(list(lookalike_results.items()), columns=["CustomerID", "Lookalikes"])

lookalike_df.head(10)

,CustomerID,Lookalikes
0,C0001,"[(C0140, 0.9969867161764148), (C0091, 0.9936678528217004), (C0069, 0.9913891295093918)]"
1,C0002,"[(C0134, 0.9988506844325151), (C0143, 0.9984231418083602), (C0159, 0.9905501434854227)]"
2,C0003,"[(C0007, 0.9977547853080485), (C0163, 0.9976764753499918), (C0005, 0.9933095942065239)]"
3,C0004,"[(C0075, 0.9943479392194652), (C0146, 0.988167716043917), (C0090, 0.975744088009604)]"
4,C0005,"[(C0163, 0.9968848831027564), (C0007, 0.9945108441120782), (C0003, 0.9933095942065239)]"
5,C0006,"[(C0185, 0.9934702881925904), (C0169, 0.9919851644958835), (C0081, 0.9901261447783123)]"
6,C0007,"[(C0163, 0.9996647685556813), (C0003, 0.9977547853080485), (C0005, 0.9945108441120782)]"
7,C0008,"[(C0055, 0.9879292831808771), (C0170, 0.975347779236153), (C0173, 0.9740580808629381)]"
8,C0009,"[(C0190, 0.9914574343850664), (C0067, 0.975676164586347), (C0049, 0.9744183865936725)]"
9,C0010,"[(C0042, 0.999988411087805), (C0083, 0.9999546343277068), (C0029, 0.9968608913367741)]"


In [11]:
lookalike_df.to_csv("Lookalike.csv", index=False)